In [81]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import html.parser as htmlParser
import re
from datetime import datetime
from langdetect import detect
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

In [5]:
np.random.seed(1)

Open data file for Charlottesville event and add some useful columns to the dataframe

In [6]:
# open file
cVille_df = pd.read_json('./Datasets/charlottesville.json')

# add ids
cVille_df['id'] = [url.rsplit('/', 1)[-1] for url in cVille_df['permalink']]

# get number of mentions instead of actual values
cVille_df['mention_count'] = [(len(mentions.split(' ')) if mentions != "" else 0) for mentions in cVille_df['mentions']]

# add hashtag count column as well
cVille_df['hashtag_count'] = [(len(hashtags.split(' ')) if hashtags != "" else 0)  for hashtags in cVille_df['hashtags'] ]

# drop username and geo
cVille_df = cVille_df.drop(['geo', 'username', 'permalink', 'mentions'], axis=1)

# add lang column
cVille_df['lang'] = [detect(text) for text in cVille_df['text']]

# drop non-english rows
cVille_df = cVille_df[cVille_df.lang == 'en']

Open data file for WomenMarch event and add some useful columns to the dataframe

In [7]:
# open file
womenMarch_df = pd.read_json('./Datasets/womenmarch.json')

# add ids
womenMarch_df['id'] = [url.rsplit('/', 1)[-1] for url in womenMarch_df['permalink']]

# get number of mentions instead of actual values
womenMarch_df['mention_count'] = [(len(mentions.split(' ')) if mentions != "" else 0) for mentions in womenMarch_df['mentions']]

# add hashtag count column as well
womenMarch_df['hashtag_count'] = [(len(hashtags.split(' ')) if hashtags != "" else 0)  for hashtags in womenMarch_df['hashtags'] ]

# drop username and geo
womenMarch_df = womenMarch_df.drop(['geo', 'username', 'permalink', 'mentions'], axis=1)

# add lang column
womenMarch_df['lang'] = [detect(text) for text in womenMarch_df['text']]

# drop non-english rows
womenMarch_df = womenMarch_df[womenMarch_df.lang == 'en']

Some more preprocessing of data.

In [8]:
def containsChar(text, pattern):
    for word in text.split(' '):
        if word == pattern: 
            return True
    return False

Preprocess data function

In [9]:
# This method preprocesses the data in given dataframe
# wasViolent = True indicates violent event and False indicates non-violent event 

def preprocessData(df, wasViolent):
    
    # convert html elements to readable format (also convert text to lowercase)
    filteredText = [htmlParser.unescape(text.lower()) for text in df['text']]

    # remove urls (because urls conatain spaces that make it difficult, this will stop at ellipsis or # or @ or end of line)
    # not 100% accurate but decent approach
    url_pattern = re.compile("http(.+?)(@\S+|#\S+|\\u2026|$)")
    filteredText = [re.sub(url_pattern, "", text) for text in filteredText]

    # remove usernames and mentions
    username_pattern = re.compile("@\S+")
    filteredText = [re.sub(username_pattern, "", text) for text in filteredText]

    # remove hashtags
    hashtag_pattern = re.compile("#\S+")
    filteredText = [re.sub(hashtag_pattern, "", text) for text in filteredText]

    # emoticon handling (replace positive_emojis with pos_emoji and negative_emojis with neg_emoji keyword)
    pos_emojis = [':-)', ':)', '(:', '(-:', ':-D', ':D', 'X-D', 'XD', 'xD', '<3', ':*', ';-)', ';)', ';-D', ';D', '(;', '(-;']
    filteredText = [' '.join([('pos_emoji' if word in pos_emojis else word) for word in text.split(' ')]) for text in filteredText]

    neg_emojis = [":-(", ":(", "):", ")-:", ":,(", ":'(", ':"(', ":(("]
    filteredText = [' '.join([('neg_emoji' if word in neg_emojis else word) for word in text.split(' ')]) for text in filteredText]

    # handle punctuations (add punc_excl=1 if ! exists else 0, same with punc_ques and ?)
    df['punc_excl'] = [(1 if containsChar(text, '!') else 0) for text in filteredText]
    df['punc_ques'] = [(1 if containsChar(text, '?') else 0) for text in filteredText]
    
    # remove pictures
    pic_pattern = re.compile("pic.twitter.com(.+)(/s|$)")
    filteredText = [re.sub(pic_pattern, "", text) for text in filteredText]

    # remove question marks and dollar signs
    removableChars = ['?', '$']
    filteredText = [' '.join([('' if word in removableChars else word) for word in text.split(' ')]) for text in filteredText]

    # remove random words (if still there is : or emoji then ignore them)
    random_pattern = re.compile("(&|:|!|,|;|)", re.I)
    filteredText = [re.sub(random_pattern, "", text) for text in filteredText]

    # handle repeating characters (replace characters repeating more than twice as two characters)
    repeated_pattern = re.compile(r"(.)\1{1,}")
    filteredText = [re.sub(repeated_pattern, r'\1\1', text) for text in filteredText]

    # add label
    df['violent'] = [1 if wasViolent else 0] * df.shape[0]

    # add filteredText column to dataframe
    df['filteredText'] = filteredText
    
    return df[['id', 'date', 'filteredText', 'hashtags', 'favorites', 'retweets', 'mention_count', 'hashtag_count', 'punc_excl', 'punc_ques', 'violent']]

Check number of valid samples in English

In [10]:
print(cVille_df.shape)
print(womenMarch_df.shape)

(153960, 9)
(109975, 9)


Preprocess both Charlottesville and WomenMarch data

In [11]:
cVille_preprocessed_df = preprocessData(cVille_df, wasViolent=True)
womenMarch_preprocessed_df = preprocessData(womenMarch_df, wasViolent=False)

In [12]:
cVille_preprocessed_df.head()

,id,date,filteredText,hashtags,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,violent
0,884925177917845504,2017-07-11,de blasio’s trip was approved for ‘city purpos...,#commies #antifa,0,2,0,2,0,0,1
1,884924281951576064,2017-07-11,alt-right trump-supporting neo-nazi confirmed.,,0,0,0,0,0,0,1
2,884923880938328066,2017-07-11,this. kills me. i remember seeing around duri...,#RobertGrodt #Occupy #BijiRojava #antifa,7,4,0,4,0,0,1
3,884923588960280576,2017-07-11,is resorting to violence and unrest to silenc...,#antifa,0,0,0,1,0,0,1
4,884922175748059136,2017-07-11,neo-nazi july 12th internet-wide day of act...,#Politics #Collusion #NetNeutrality,0,1,1,3,0,0,1


In [13]:
womenMarch_preprocessed_df.head()

,id,date,filteredText,hashtags,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,violent
0,921443913398972416,2017-10-20,can we make our message any clearer?,#MeToo #EndRapeCulture #NotOkay #WhyIMarch,0,1,0,4,0,0,0
1,921204515738546176,2017-10-19,anyone who doesn't support affordable is not ...,#birthcontrol #prolife #ProofIsInThePudding #W...,1,1,0,5,0,0,0
2,921100053720256514,2017-10-19,each and every day for,#WhyIMarch #RacialJustice #M4RJ #BlackLivesMatter,0,2,0,4,0,0,0
3,920843500781953024,2017-10-18,because deserve a livable future.,#whyimarch #youth,0,1,1,2,0,0,0
4,920765197605855232,2017-10-18,started a conversation with my daughter . i ...,#MeToo #WhyIMarch #ShePersisted,2,3,1,3,0,0,0


In [14]:
# write preprocessed data back to to the disk
cVille_preprocessed_df.to_csv('./Datasets/charlottesville_preprocessed.csv', sep='\t', index=False)
womenMarch_preprocessed_df.to_csv('./Datasets/womenMarch_preprocessed.csv', sep='\t', index=False)

Read Charlottesville and Women March preprocessed data from the disk

In [15]:
cVille_preprocessed_df = pd.read_csv('./Datasets/charlottesville_preprocessed.csv', sep='\t')
womenMarch_preprocessed_df = pd.read_csv('./Datasets/womenMarch_preprocessed.csv', sep='\t')

Handle NaN values. Replace them with empty string in filteredText column

In [16]:
cVille_preprocessed_df.info()
cVille_preprocessed_df['filteredText'].fillna("", inplace=True)
cVille_preprocessed_df['hashtags'].fillna("", inplace=True)
cVille_preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153960 entries, 0 to 153959
Data columns (total 11 columns):
id               153960 non-null int64
date             153960 non-null object
filteredText     153898 non-null object
hashtags         143925 non-null object
favorites        153960 non-null int64
retweets         153960 non-null int64
mention_count    153960 non-null int64
hashtag_count    153960 non-null int64
punc_excl        153960 non-null int64
punc_ques        153960 non-null int64
violent          153960 non-null int64
dtypes: int64(8), object(3)
memory usage: 12.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153960 entries, 0 to 153959
Data columns (total 11 columns):
id               153960 non-null int64
date             153960 non-null object
filteredText     153960 non-null object
hashtags         153960 non-null object
favorites        153960 non-null int64
retweets         153960 non-null int64
mention_count    153960 non-null int64
hashtag_count    15

In [17]:
womenMarch_preprocessed_df.info()
womenMarch_preprocessed_df['hashtags'].fillna("", inplace=True)
womenMarch_preprocessed_df['filteredText'].fillna("", inplace=True)
womenMarch_preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109975 entries, 0 to 109974
Data columns (total 11 columns):
id               109975 non-null int64
date             109975 non-null object
filteredText     109835 non-null object
hashtags         109975 non-null object
favorites        109975 non-null int64
retweets         109975 non-null int64
mention_count    109975 non-null int64
hashtag_count    109975 non-null int64
punc_excl        109975 non-null int64
punc_ques        109975 non-null int64
violent          109975 non-null int64
dtypes: int64(8), object(3)
memory usage: 9.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109975 entries, 0 to 109974
Data columns (total 11 columns):
id               109975 non-null int64
date             109975 non-null object
filteredText     109975 non-null object
hashtags         109975 non-null object
favorites        109975 non-null int64
retweets         109975 non-null int64
mention_count    109975 non-null int64
hashtag_count    109

In [18]:
womenMarch_preprocessed_df.head()

,id,date,filteredText,hashtags,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,violent
0,921443913398972416,2017-10-20,can we make our message any clearer?,#MeToo #EndRapeCulture #NotOkay #WhyIMarch,0,1,0,4,0,0,0
1,921204515738546176,2017-10-19,anyone who doesn't support affordable is not ...,#birthcontrol #prolife #ProofIsInThePudding #W...,1,1,0,5,0,0,0
2,921100053720256514,2017-10-19,each and every day for,#WhyIMarch #RacialJustice #M4RJ #BlackLivesMatter,0,2,0,4,0,0,0
3,920843500781953024,2017-10-18,because deserve a livable future.,#whyimarch #youth,0,1,1,2,0,0,0
4,920765197605855232,2017-10-18,started a conversation with my daughter . i ...,#MeToo #WhyIMarch #ShePersisted,2,3,1,3,0,0,0


In [19]:
# below is part of preprocessing
def feature_reduced_text(text, stop_words, lemmatizer):
    # tokenization
    tokens = word_tokenize(text)

    # remove stopwords from text
    tokens = [token for token in tokens if token not in stop_words]

    # lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # pos tagging
    pos_tags = nltk.pos_tag(tokens)
    
    # return only relevant words (adjective, verb, adverb, noun with all of their variations)
    return ' '.join([tag[0] for tag in pos_tags if tag[1] in ['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'NNP', 'NNPS', 'RB', 'RBR', 'RBS', 'VB', 'VBG', 'VBD', 'VBN', 'VBP', 'VBZ']])

In [20]:
stops = set(stopwords.words('english'))
# We will use not for negation (may not be perfect but decent enough)
stops.remove('not')
word_lemmatizer = WordNetLemmatizer()

cVille_preprocessed_df['preprocessedText'] = [feature_reduced_text(text, stop_words=stops, lemmatizer=word_lemmatizer) for text in cVille_preprocessed_df['filteredText'].tolist()]
womenMarch_preprocessed_df['preprocessedText'] = [feature_reduced_text(text, stop_words=stops, lemmatizer=word_lemmatizer) for text in womenMarch_preprocessed_df['filteredText'].tolist()]

Start Data transformation (negation handling, sentiment/polarity score and feature extraction)

In [21]:
# Negation handling (replace (isn't, ain't, haven't etc with not. stop words don't matter anyways))

def getNegationHandledText(text):
    negation_pattern = re.compile(r"(\w*)n't ")
    cleanedText = re.sub(negation_pattern, 'not ', text)
    
    # phrase occuring after not (combine all sub phrases to one phrase)
    negationSplit = cleanedText.split(' not ')
    positivePhrase = negationSplit[0]
    negativePhrase = ' '.join(negationSplit[1:])
                       
    # return words with _NEG appended to them
    if negativePhrase:
        return ' '.join([positivePhrase, ' '.join([word + "_NEG" for word in negativePhrase.split(' ') if word != " "])])
    else:
        return text

In [22]:
# add negation handled text to Charlottesville df
cVille_preprocessed_df['negationText'] = [getNegationHandledText(text) for text in cVille_preprocessed_df['preprocessedText']]

# add negation handled text to Women March df
womenMarch_preprocessed_df['negationText'] = [getNegationHandledText(text) for text in womenMarch_preprocessed_df['preprocessedText']]

In [23]:
# Get sentiment score for hashtag sentences
def hashtag_sentiment_score(hashtags):
    # Convert hashtag to words/sentences
    def hashtagToSentence(hashtag):
        return " ".join([word for word in re.split('([A-Z][a-z]+)', hashtag) if word])
    
    scores = [vaderAnalyzer.polarity_scores(hashtagToSentence(hashtag))['compound'] for hashtag in hashtags]
    return scores

def sentiment_score(tweet, analyzer):
    hashtag_scores = hashtag_sentiment_score(tweet['hashtags'].split())
    text_sentiment_score = analyzer.polarity_scores(tweet['preprocessedText'])['compound']
    
    normalized_score = (np.sum(hashtag_scores) + text_sentiment_score) / (len(hashtag_scores) + 1)
    return normalized_score

Calculate sentiment score/polarity calculation. Here, we use two methods to calculate polarity scores.
1. Vader Sentiment Intensity Analyzer that takes care of negation and is built for messy social media data like Twitter
2. Our approach to negate the polarities of words existing after not (not itself is not considered in this case)

In [24]:
# Part 1 - Calculate Vader score for text
vaderAnalyzer = SentimentIntensityAnalyzer()

cVille_preprocessed_df['vaderTextScore'] = [vaderAnalyzer.polarity_scores(text)['compound'] for text in cVille_preprocessed_df['preprocessedText']]
womenMarch_preprocessed_df['vaderTextScore'] = [vaderAnalyzer.polarity_scores(text)['compound'] for text in womenMarch_preprocessed_df['preprocessedText']]

# Part 2 - Calculate _NEG handled score

# Calculate normalized sentiment score with Vader (both text and hashtags)
cVille_preprocessed_df['vaderScore'] = [sentiment_score(row, vaderAnalyzer) for index, row in cVille_preprocessed_df.iterrows()]
womenMarch_preprocessed_df['vaderScore'] = [sentiment_score(row, vaderAnalyzer) for index, row in womenMarch_preprocessed_df.iterrows()]

In [25]:
cVille_preprocessed_df.drop(['negationText'], axis=1, inplace=True)
womenMarch_preprocessed_df.drop(['negationText'], axis=1, inplace=True)

cVille_preprocessed_df.head()

,id,date,filteredText,hashtags,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,violent,preprocessedText,vaderTextScore,vaderScore
0,884925177917845504,2017-07-11,de blasio’s trip was approved for ‘city purpos...,#commies #antifa,0,2,0,2,0,0,1,’ trip approved ‘ city purpose ’ bashing trump...,0.4215,0.140500
1,884924281951576064,2017-07-11,alt-right trump-supporting neo-nazi confirmed.,,0,0,0,0,0,0,1,alt-right trump-supporting neo-nazi confirmed,0.0000,0.000000
2,884923880938328066,2017-07-11,this. kills me. i remember seeing around duri...,#RobertGrodt #Occupy #BijiRojava #antifa,7,4,0,4,0,0,1,kill remember seeing around rest power comrade,-0.6908,-0.138160
3,884923588960280576,2017-07-11,is resorting to violence and unrest to silenc...,#antifa,0,0,0,1,0,0,1,resorting violence unrest silence opposition f...,-0.8271,-0.413550
4,884922175748059136,2017-07-11,neo-nazi july 12th internet-wide day of act...,#Politics #Collusion #NetNeutrality,0,1,1,3,0,0,1,neo-nazi july internet-wide day action save,0.4939,0.123475


In [26]:
womenMarch_preprocessed_df.head()

,id,date,filteredText,hashtags,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,violent,preprocessedText,vaderTextScore,vaderScore
0,921443913398972416,2017-10-20,can we make our message any clearer?,#MeToo #EndRapeCulture #NotOkay #WhyIMarch,0,1,0,4,0,0,0,make message clearer,0.0000,-0.172060
1,921204515738546176,2017-10-19,anyone who doesn't support affordable is not ...,#birthcontrol #prolife #ProofIsInThePudding #W...,1,1,0,5,0,0,0,anyone n't support affordable not,-0.3089,-0.051483
2,921100053720256514,2017-10-19,each and every day for,#WhyIMarch #RacialJustice #M4RJ #BlackLivesMatter,0,2,0,4,0,0,0,day,0.0000,0.110500
3,920843500781953024,2017-10-18,because deserve a livable future.,#whyimarch #youth,0,1,1,2,0,0,0,deserve livable future,0.0000,0.000000
4,920765197605855232,2017-10-18,started a conversation with my daughter . i ...,#MeToo #WhyIMarch #ShePersisted,2,3,1,3,0,0,0,started conversation daughter pray never exper...,0.6310,0.157750


In [27]:
def getDays(dateStrings, finalDateStr):
    dateFormat = '%Y-%m-%d'
    
    def dateFromStr(dStr):
        return datetime.strptime(dStr, dateFormat)
    
    finalDate = dateFromStr(finalDateStr)   
    return [(finalDate - dateFromStr(dateStr)).days for dateStr in dateStrings]

Remove entries which are during the events as they dont help in predicting (discuss with the team)

In [28]:
# get days
cVille_preprocessed_df['days_to_event'] = getDays(cVille_preprocessed_df['date'], '2017-08-11')
womenMarch_preprocessed_df['days_to_event'] = getDays(womenMarch_preprocessed_df['date'], '2017-01-21')

# get relevant data
# cVille_preprocessed_relevant_df = cVille_preprocessed_df[(cVille_preprocessed_df.date != '2017-08-11') & (cVille_preprocessed_df.date != '2017-08-12')]
# womenMarch_preprocessed_relevant_df = womenMarch_preprocessed_df[(womenMarch_preprocessed_df.date != '2017-01-21') & (womenMarch_preprocessed_df.date != '2017-01-22')]

cVille_preprocessed_relevant_df = cVille_preprocessed_df[(cVille_preprocessed_df.days_to_event <= 60) & (cVille_preprocessed_df.days_to_event > 0)]
womenMarch_preprocessed_relevant_df = womenMarch_preprocessed_df[(womenMarch_preprocessed_df.days_to_event <= 60) & (womenMarch_preprocessed_df.days_to_event > 0)]

In [ ]:
# import collections
# collections.Counter(womenMarch_preprocessed_relevant_df['date'])

Feature Extraction

1. Tf-idf vectorizer
2. Subset relevant features in a data frame

In [29]:
cVille_features_df = cVille_preprocessed_relevant_df[['id', 'date', 'days_to_event', 'preprocessedText', 'favorites', 'retweets', 'mention_count', 'hashtag_count', 'punc_excl', 'punc_ques', 'vaderTextScore', 'vaderScore', 'violent']]
womenMarch_features_df = womenMarch_preprocessed_relevant_df[['id', 'date', 'days_to_event', 'preprocessedText', 'favorites', 'retweets', 'mention_count', 'hashtag_count', 'punc_excl', 'punc_ques', 'vaderTextScore', 'vaderScore', 'violent']]

cVille_features_df.info()
womenMarch_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30627 entries, 0 to 153959
Data columns (total 13 columns):
id                  30627 non-null int64
date                30627 non-null object
days_to_event       30627 non-null int64
preprocessedText    30627 non-null object
favorites           30627 non-null int64
retweets            30627 non-null int64
mention_count       30627 non-null int64
hashtag_count       30627 non-null int64
punc_excl           30627 non-null int64
punc_ques           30627 non-null int64
vaderTextScore      30627 non-null float64
vaderScore          30627 non-null float64
violent             30627 non-null int64
dtypes: float64(2), int64(9), object(2)
memory usage: 3.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29754 entries, 40111 to 109974
Data columns (total 13 columns):
id                  29754 non-null int64
date                29754 non-null object
days_to_event       29754 non-null int64
preprocessedText    29754 non-null object
favorites

In [30]:
training_features_df = pd.concat([cVille_features_df, womenMarch_features_df])
training_features_df.head()

,id,date,days_to_event,preprocessedText,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,vaderTextScore,vaderScore,violent
0,884925177917845504,2017-07-11,31,’ trip approved ‘ city purpose ’ bashing trump...,0,2,0,2,0,0,0.4215,0.140500,1
1,884924281951576064,2017-07-11,31,alt-right trump-supporting neo-nazi confirmed,0,0,0,0,0,0,0.0000,0.000000,1
2,884923880938328066,2017-07-11,31,kill remember seeing around rest power comrade,7,4,0,4,0,0,-0.6908,-0.138160,1
3,884923588960280576,2017-07-11,31,resorting violence unrest silence opposition f...,0,0,0,1,0,0,-0.8271,-0.413550,1
4,884922175748059136,2017-07-11,31,neo-nazi july internet-wide day action save,0,1,1,3,0,0,0.4939,0.123475,1


In [31]:
training_features_df.shape

(60381, 13)

In [32]:
# use tf-idf vectorizer
tv = TfidfVectorizer(stop_words='english', min_df=0.0005)
tfidf_features = tv.fit_transform(training_features_df['preprocessedText'])

In [33]:
tfidf_features.shape

(60381, 1857)

In [ ]:
# tv.vocabulary_

In [34]:
# create pandas dataframe from tfidf features
tfidf_features_df = pd.DataFrame(tfidf_features.toarray())

In [35]:
# reset indices so that they can be concatenated into one dataframe
temp_training_features_df = training_features_df.reset_index(drop=True)
temp_tfidf_features_df = tfidf_features_df.reset_index(drop=True)

final_training_features_df = pd.concat([temp_training_features_df, temp_tfidf_features_df], axis=1)

In [36]:
final_training_features_df.shape

(60381, 1870)

In [37]:
final_training_features_df.head()

,id,date,days_to_event,preprocessedText,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,...,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856
0,884925177917845504,2017-07-11,31,’ trip approved ‘ city purpose ’ bashing trump...,0,2,0,2,0,0,...,0.0,0.0,0.0,0.449902,0.0,0.0,0.0,0.0,0.0,0.0
1,884924281951576064,2017-07-11,31,alt-right trump-supporting neo-nazi confirmed,0,0,0,0,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,884923880938328066,2017-07-11,31,kill remember seeing around rest power comrade,7,4,0,4,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,884923588960280576,2017-07-11,31,resorting violence unrest silence opposition f...,0,0,0,1,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,884922175748059136,2017-07-11,31,neo-nazi july internet-wide day action save,0,1,1,3,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# drop id, date and preprocessedText from final training set
final_training_features_df.drop(['id', 'date', 'preprocessedText'], axis=1, inplace=True)

In [39]:
final_training_features_df.head()

,days_to_event,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,vaderTextScore,vaderScore,violent,...,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856
0,31,0,2,0,2,0,0,0.4215,0.140500,1,...,0.0,0.0,0.0,0.449902,0.0,0.0,0.0,0.0,0.0,0.0
1,31,0,0,0,0,0,0,0.0000,0.000000,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,31,7,4,0,4,0,0,-0.6908,-0.138160,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,31,0,0,0,1,0,0,-0.8271,-0.413550,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,31,0,1,1,3,0,0,0.4939,0.123475,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
final_training_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60381 entries, 0 to 60380
Columns: 1867 entries, days_to_event to 1856
dtypes: float64(1859), int64(8)
memory usage: 860.1 MB


Data Exploration and Visualization

In [90]:
final_training_features_df.describe()

,days_to_event,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,vaderTextScore,vaderScore,violent,...,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856
count,60381.000000,60381.000000,60381.000000,60381.000000,60381.000000,60381.000000,60381.000000,60381.000000,60381.000000,60381.000000,...,60381.000000,60381.000000,60381.000000,60381.000000,60381.000000,60381.000000,60381.000000,60381.00000,60381.000000,60381.000000
mean,20.639026,17.851162,8.535152,0.387109,2.132558,0.018797,0.014259,0.014519,-0.008952,0.507229,...,0.002826,0.000693,0.000310,0.000396,0.002771,0.000806,0.000360,0.00073,0.000250,0.000307
std,19.354121,270.036685,116.589798,0.829540,1.782515,0.135810,0.118559,0.420856,0.223970,0.499952,...,0.042961,0.018535,0.012762,0.014784,0.036733,0.020913,0.014047,0.01807,0.011309,0.011931
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.980500,-0.967000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,3.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-0.205700,-0.096875,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,14.000000,2.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
75%,36.000000,5.000000,2.000000,1.000000,3.000000,0.000000,0.000000,0.318200,0.101150,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
max,60.000000,25679.000000,9456.000000,14.000000,24.000000,1.000000,1.000000,0.997200,0.955200,1.000000,...,1.000000,0.774777,0.730130,0.952869,0.735571,1.000000,0.803454,1.00000,0.864722,0.696537


In [91]:
final_training_features_df.groupby(['days_to_event', 'violent'])['vaderScore'].mean()

days_to_event  violent
1              0          0.049725
               1         -0.025878
2              0          0.046645
               1         -0.043971
3              0          0.053189
               1         -0.039625
4              0          0.053704
               1         -0.042802
5              0          0.058626
               1         -0.039933
6              0          0.052598
               1         -0.071084
7              0          0.051603
               1         -0.042247
8              0          0.044554
               1         -0.034448
9              0          0.038151
               1         -0.039676
10             0          0.057710
               1         -0.050518
11             0          0.035761
               1         -0.072941
12             0          0.069736
               1         -0.053871
13             0          0.079391
               1         -0.069405
14             0          0.036643
               1         -0.0684

In [ ]:
test_df = final_training_features_df[['days_to_event', 'vaderScore', 'violent']]
test_df = test_df.sort_values(by='days_to_event')

days = test_df['days_to_event']
violentSc = test_df[test_df.violent == 1]
nonSc = test_df[test_df.violent == 0]

In [ ]:
plt.hist(violentSc, days, alpha=0.5, label='x')
plt.hist(nonSc, days, alpha=0.5, label='y')
plt.legend(loc='upper right')
plt.show()

Split Train data and Test data 

In [41]:
# shuffle dataset
final_training_features_df = final_training_features_df.sample(frac=1, random_state=np.random.seed(1))
final_training_features_df.head()

,days_to_event,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,vaderTextScore,vaderScore,violent,...,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856
42325,12,3,0,1,2,0,0,0.7506,0.250200,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36808,2,0,1,0,2,0,0,0.5390,0.179667,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19271,3,1,0,0,2,0,0,0.0000,0.000000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11738,53,1,0,1,1,0,0,0.0000,0.000000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43525,18,5,0,0,1,0,0,-0.3182,-0.159100,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
final_training_features_df.tail()

,days_to_event,favorites,retweets,mention_count,hashtag_count,punc_excl,punc_ques,vaderTextScore,vaderScore,violent,...,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856
50057,2,3,4,1,3,0,0,0.4939,0.123475,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32511,1,1,0,0,1,0,0,0.4939,0.246950,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5192,38,0,0,0,4,0,0,0.5859,0.242160,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12172,54,3,0,0,6,0,0,-0.6581,-0.094014,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33003,1,6,0,0,2,0,0,0.0000,0.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
Y = final_training_features_df['violent'] 
X = final_training_features_df.loc[:, final_training_features_df.columns != 'violent']

In [44]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60381 entries, 42325 to 33003
Columns: 1866 entries, days_to_event to 1856
dtypes: float64(1859), int64(7)
memory usage: 860.1 MB


In [45]:
Y.head()

42325    0
36808    0
19271    1
11738    1
43525    0
Name: violent, dtype: int64

Train test split

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=np.random.seed(1))

In [47]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(48304, 1866)
(48304,)
(12077, 1866)
(12077,)


Testing with imbalanced data

Calculate baseline

In [48]:
print(np.mean(Y_train))

0.506997350116


seems like around 50.7% accuracy could be achieved by simply choosing majority class (not violent).

Logistic Regression

In [49]:
logistic_clf = LogisticRegression()
logistic_score = cross_val_score(logistic_clf, X_train, Y_train, scoring='accuracy', cv=5)
print(np.mean(logistic_score))

0.924995916432


Our model is better than baseline as it is 92.5% accurate without tuning hyperparameters.

Naive Bayes' Model/Classifier

In [75]:
nb_clf = BernoulliNB()
nb_score = cross_val_score(nb_clf, X_train, Y_train, scoring='accuracy', cv=5)
print(np.mean(nb_score))

0.870238636356


Our model is better than baseline as it gives around 87% accuracy without tuning hyperparameters.

SVM Classifier

In [ ]:
# svm_clf = SVC()
# svm_score = cross_val_score(svm_clf, X_train, Y_train, scoring='accuracy', cv=5)
# print(np.mean(svm_score))

Neural Network Classifier

In [51]:
from sklearn.neural_network import MLPClassifier

nn_clf = MLPClassifier()
nn_score = cross_val_score(nn_clf, X_train, Y_train, scoring='accuracy', cv=5)
print(np.mean(nn_score))

0.930295712008


Our model is clearly better than baseline as it gives above 93% accuracy without tuning hyperparameters.

Final Accuracy Test

In [85]:
def evaluate(classifier, xTrain, yTrain, xTest, yTest):
    classifier.fit(xTrain, yTrain)
    yPred = classifier.predict(xTest)
    accuracy = accuracy_score(yTest, yPred)
    confusionMat = confusion_matrix(yTest, yPred)
    f1 = f1_score(yTest, yPred)
    return (accuracy, confusionMat, f1)

Accuracy prediction on test data

In [86]:
final_logistic_clf = LogisticRegression()
final_nb_clf = BernoulliNB()
final_nn_clf = MLPClassifier()

logistic_evaluation = evaluate(final_logistic_clf, X_train, Y_train, X_test, Y_test)
nb_evaluation = evaluate(final_nb_clf, X_train, Y_train, X_test, Y_test)
nn_evaluation = evaluate(final_nn_clf, X_train, Y_train, X_test, Y_test)

In [87]:
# Logistic regression accuracy
print("Logistic Regression Accuracy:\n{}\n".format(logistic_evaluation[0]))
print("Logistic Regression Confusion Matrix:\n{}\n".format(logistic_evaluation[1]))
print("Logistic Regression F1 Score:\n", logistic_evaluation[2])

Logistic Regression Accuracy:
0.9234081311584003

Logistic Regression Confusion Matrix:
[[5609  331]
 [ 594 5543]]

Logistic Regression F1 Score:
 0.922987261677


Logistic Regression Model gives 92.3% accuracy with 0.923 f1 score

In [88]:
# Neural Network accuracy
print("Neural Network Accuracy:\n{}\n".format(nn_evaluation[0]))
print("Neural Network Confusion Matrix:\n{}\n".format(nn_evaluation[1]))
print("Neural Network F1 score:\n", nn_evaluation[2])

Neural Network Accuracy:
0.9350004140101018

Neural Network Confusion Matrix:
[[5594  346]
 [ 439 5698]]

Neural Network F1 score:
 0.935555373122


Neural Network gives 93.5% accuracy with 0.935 f1 score

In [89]:
# Naive Bayes' accuracy
print("Naive Bayes' Accuracy:\n{}\n".format(nb_evaluation[0]))
print("Naive Bayes' Confusion Matrix:\n{}\n".format(nb_evaluation[1]))
print("Naive Bayes' F1 Score:", nb_evaluation[2])

Naive Bayes' Accuracy:
0.8706632441831581

Naive Bayes' Confusion Matrix:
[[5590  350]
 [1212 4925]]

Naive Bayes' F1 Score: 0.863126533474


Naive Bayes' Model gives 87% accuracy with 0.863 f1 score